# GANomaly Notebook Results

## Initial Configurations

### Libraries import

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import umap
import cv2
from IPython.display import display, clear_output
from ipywidgets import interact, IntSlider

sys.path.append("../../")

In [2]:
from utils.metrics import shapiroWilks_test, dagostinoPearson_test, bartlett_test, levene_test, fOneWay_test
from utils.savers import generate_qq_plot

### Experiment selection

In [3]:
experiment_id = "0010"
root_path = "/home/jefelitman/Saved_Models/Anomaly_parkinson/"
for i in sorted(os.listdir("/home/jefelitman/Saved_Models/Anomaly_parkinson/")):
    if experiment_id in i:
        experiment_folder = os.path.join(root_path, i)
experiment_folder

'/home/jefelitman/Saved_Models/Anomaly_parkinson/0010_Ganomaly_2D-256x256x64'

## Quantitative metrics

### Metrics loading

In [4]:
train_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/train.csv"))
data = train_metrics
np.max(data, axis=0), np.min(data, axis=0), data.tail()

(epoch          29999.000000
 gen_error        660.663879
 disc_error         7.817431
 accuracy           0.930556
 precision          0.000000
 recall                  NaN
 specificity        0.930556
 f1_score           0.000000
 auc                0.000000
 dtype: float64,
 epoch          0.000000
 gen_error      0.590776
 disc_error     2.942470
 accuracy       0.291667
 precision      0.000000
 recall              NaN
 specificity    0.291667
 f1_score       0.000000
 auc            0.000000
 dtype: float64,
 array([2.99990000e+04, 6.23744249e-01, 7.62461948e+00, 6.52777791e-01,
        0.00000000e+00,            nan, 6.52777791e-01, 0.00000000e+00,
        0.00000000e+00]))

In [6]:
test_metrics = pd.read_csv(os.path.join(experiment_folder, "metrics/test.csv"))
data = test_metrics
np.max(data, axis=0), np.min(data, axis=0), data.tail()

(epoch          29999.000000
 accuracy           0.855769
 precision          1.000000
 recall             0.829545
 specificity        1.000000
 f1_score           0.906832
 auc                1.000000
 dtype: float64,
 epoch          0.000000
 accuracy       0.067308
 precision      0.111111
 recall         0.011364
 specificity    0.000000
 f1_score       0.020619
 auc            0.009943
 dtype: float64,
        epoch  accuracy  precision    recall  specificity  f1_score       auc
 29995  29995  0.605769        1.0  0.534091          1.0  0.696296  1.000000
 29996  29996  0.644231        1.0  0.579545          1.0  0.733813  0.991832
 29997  29997  0.567308        1.0  0.488636          1.0  0.656489  1.000000
 29998  29998  0.653846        1.0  0.590909          1.0  0.742857  0.994318
 29999  29999  0.625000        1.0  0.556818          1.0  0.715328  0.994318)

### Train Graphics

In [7]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 12
columns = 5
for metric in ["gen_error", "disc_error", "accuracy", "specificity"]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(train_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'train_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

### Test graphics

In [8]:
path = os.path.join(experiment_folder, "outputs/graphics/quantitative/")
portions = 500
rows = 12
columns = 5
for metric in test_metrics.columns[1:]:
    fig, axs = plt.subplots(rows, columns, figsize=(30, 25))
    fig.suptitle('{} over Epochs'.format(metric))
    for i in range(rows):
        for j in range(columns):
            axs[i, j].plot(test_metrics[metric][(i*columns+j)*portions:(i*columns+j+1)*portions])
            filename = 'test_{}.png'.format(metric)
    fig.savefig(path+filename)
    plt.close(fig)

## Qualitative metrics

### Errors loading

In [9]:
base_path = os.path.join(experiment_folder, "outputs/errors/")
for t in ["encoder", "contextual", "adversarial"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = np.r_[[]]
            
            errors = np.load(os.path.join(base_path, t, m, c+".npy"))
            globals()["{}_{}_{}".format(m, t, c)] = errors
            globals()[all_data] = np.concatenate([globals()[all_data], errors])

### Train means and stds

In [10]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        data = globals()["train_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0005634856750273482 +- 0.00030529587669815783

contextual error (normal): 0.01207544998679724 +- 0.0017062103510774049

adversarial error (normal): 0.018182152274271682 +- 0.005344321682463277



### Train statistical tests

In [11]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        print("-------------------------------------------------")
        data = globals()["train_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 0.003501273764339694
Shapiro-Wilks test for encoder (normal): 0.0003386996977496892
Barlett test for encoder (normal): 1.4190204114689143e-228
Levene test for encoder (normal): 3.285423233662101e-22
F1 test for encoder (normal): 0.6244774456095635
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.03813721381714772
Shapiro-Wilks test for contextual (normal): 0.006934409029781818
Barlett test for contextual (normal): 9.0721367117138e-175
Levene test for contextual (normal): 8.309428763581047e-21
F1 test for contextual (normal): 0.21540922536466797
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for adversarial (normal): 0.007582750746299013
Shapiro-Wilks test for adversarial (normal): 3.836040559690446e-05
Barlett tes

### Test means and stds

In [12]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["test_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0009448078462810372 +- 0.0004947018925274825
encoder error (abnormal): 0.2359988685189323 +- 0.08662532752428148

contextual error (normal): 0.01193352317204699 +- 0.001260851146065392
contextual error (abnormal): 0.1929209144278006 +- 0.04577405023818982

adversarial error (normal): 0.020554132701363415 +- 0.007025139944602236
adversarial error (abnormal): 0.8737956065345894 +- 0.33418175548036283



### Test statistical tests

In [14]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["test_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 0.6212754693632945
Shapiro-Wilks test for encoder (normal): 0.5110328197479248
Barlett test for encoder (normal): 9.907601504624316e-46
Levene test for encoder (normal): 3.411273451665428e-10
F1 test for encoder (normal): 0.23842314004259743
-------------------------------------------------
-------------------------------------------------
D'agostino-Pearson test for encoder (abnormal): 0.003626524972103315
Shapiro-Wilks test for encoder (abnormal): 6.631091309827752e-06
Barlett test for encoder (abnormal): 5.453108282552607e-69
Levene test for encoder (abnormal): 2.8114704879457886e-21
F1 test for encoder (abnormal): 0.041382934326941484
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.2109994973979376
Shapiro-Wilks test for contextual (normal): 0.15943466126918793
Barlett test for context

### All data means and stds

In [15]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        data = globals()["all_{}_{}".format(t,c)]
        print("{i} error ({j}): {m} +- {s}".format(
            i = t,
            j = c,
            m = np.mean(data),
            s = np.std(data)
        ))
    print("")

encoder error (normal): 0.0006328169788916553 +- 0.0003773409533503786
encoder error (abnormal): 0.2359988685189323 +- 0.08662532752428148

contextual error (normal): 0.012049645111388103 +- 0.0016352046499146817
contextual error (abnormal): 0.1929209144278006 +- 0.04577405023818982

adversarial error (normal): 0.018613421442833813 +- 0.005760112127483862
adversarial error (abnormal): 0.8737956065345894 +- 0.33418175548036283



### All data statistical tests

In [16]:
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        print("-------------------------------------------------")
        data = globals()["all_{}_{}".format(t,c)]
        norm_dist = stats.norm.rvs(loc=0, scale=1, size=data.shape)
        print("D'agostino-Pearson test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = dagostinoPearson_test(data)
        ))
        print("Shapiro-Wilks test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = shapiroWilks_test(data)
        ))
        print("Barlett test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = bartlett_test(data, norm_dist)
        ))
        print("Levene test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = levene_test(data, norm_dist)
        ))
        print("F1 test for {i} ({j}): {m}".format(
            i = t,
            j = c,
            m = fOneWay_test(data, norm_dist)
        ))
        print("-------------------------------------------------")
    print("")

-------------------------------------------------
D'agostino-Pearson test for encoder (normal): 3.240295364411539e-05
Shapiro-Wilks test for encoder (normal): 6.965923148527509e-06
Barlett test for encoder (normal): 3.700169203465865e-269
Levene test for encoder (normal): 4.2271662075612e-24
F1 test for encoder (normal): 0.7187395884344359
-------------------------------------------------
-------------------------------------------------
D'agostino-Pearson test for encoder (abnormal): 0.003626524972103315
Shapiro-Wilks test for encoder (abnormal): 6.631091309827752e-06
Barlett test for encoder (abnormal): 1.8259942449093752e-69
Levene test for encoder (abnormal): 1.461788416269539e-21
F1 test for encoder (abnormal): 0.06254602667137028
-------------------------------------------------

-------------------------------------------------
D'agostino-Pearson test for contextual (normal): 0.021491927241864828
Shapiro-Wilks test for contextual (normal): 0.0031588124111294746
Barlett test for 

### Train graphics

#### QQ plots

In [17]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal"]:
        generate_qq_plot(globals()["train_{}_{}".format(t,c)], save_path, "train_{}_{}".format(t,c))

### Test graphics

#### Box plots

In [18]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "test_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'test_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [19]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["test_{}_{}".format(t,c)], save_path, "test_{}_{}".format(t,c))

### All data graphics

#### Box plots

In [20]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
rows = 1
columns = 3
fig, axs = plt.subplots(rows, columns, figsize=(15, 10))
for i, t in enumerate(["encoder", "contextual", "adversarial"]):
    data = "all_{}_".format(t)
    axs[i].boxplot([globals()[data+"normal"], globals()[data+"abnormal"]], labels=['Normal', 'Abnormal'])
    axs[i].set_title("{} errors".format(t))

filename = 'all_data_boxplot.png'
fig.savefig(save_path+filename)
plt.close(fig)

#### QQ plots

In [21]:
save_path = os.path.join(experiment_folder, "outputs/graphics/qualitative/")
for t in ["encoder", "contextual", "adversarial"]:
    for c in ["normal", "abnormal"]:
        generate_qq_plot(globals()["all_{}_{}".format(t,c)], save_path, "all_data_{}_{}".format(t,c))

## Visual results

### UMAP latent vectors

In [22]:
base_path = os.path.join(experiment_folder, "outputs/latent_vectors/")
for t in ["in", "out"]:
    for m in ["train", "test"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        for c in classes:
            all_data = "all_{}_{}".format(t, c)
            if all_data not in globals().keys():
                globals()[all_data] = []
            
            data = "{}_{}_{}".format(m, t, c)
            globals()[data] = []
            path = os.path.join(base_path, t, m, c)
            for file in sorted(os.listdir(path)):
                vector = np.load(os.path.join(path, file))
                globals()[data].append(vector)
                globals()[all_data].append(vector)
            globals()[data] = np.r_[globals()[data]]
for t in ["in", "out"]:
    for c in ["normal", "abnormal"]:
        all_data = "all_{}_{}".format(t, c)
        globals()[all_data] = np.r_[globals()[all_data]]

In [23]:
save_path = os.path.join(experiment_folder, "outputs/graphics/visuals/")
for t in ["in", "out"]:
    for m in ["train", "test", "all"]:
        classes = ["normal"] if m == "train" else ["normal", "abnormal"]
        globals()["{}_mapped_{}".format(m, t)] = umap.UMAP(random_state = 8128).fit_transform(np.concatenate(
                [globals()["{}_{}_{}".format(m, t, c)] for c in classes], axis=0
            )
        )
        
        divisor = globals()["{}_{}_normal".format(m, t)].shape[0]
        plt.scatter(globals()["{}_mapped_{}".format(m, t)][:divisor,0],
            globals()["{}_mapped_{}".format(m, t)][:divisor,1], color="blue", s=5, label="Normal"
        )
        if "abnormal" in classes:
            plt.scatter(globals()["{}_mapped_{}".format(m, t)][divisor:,0],
                globals()["{}_mapped_{}".format(m, t)][divisor:,1], color="red", s=5, label="Abnormal"
            )
        plt.legend()
        plt.title("UMAP for {}put data in {}".format(t, m))
        filename = '{}_umap_{}put.png'.format(m, t)
        plt.savefig(save_path+filename)
        plt.close()

### Video comparision

In [ ]:
def show_all_data(volumes):
    frame_slider = IntSlider(value=32, min=1, max=volumes[0].shape[0], step=1)
    volume_slider = IntSlider(min=1, max=len(volumes), step=1)

    def update_frame_max(*args):
        frame_slider.max = len(sorted(os.listdir(path)))
    volume_slider.observe(update_frame_max, 'value')

    interact(lambda volume, frame: plt.imshow(volumes[volume-1][frame-1,:,:,1], cmap="gray"), 
             volume=volume_slider, frame=frame_slider
    );
show_all_data(classes[0])

In [25]:
for i in range(0, 88):
    video_index = i
    mode = "test"
    video_class = "abnormal"

    base_path = os.path.join(experiment_folder, "outputs/samples/")
    for t in ["real", "fake", "substraction"]:
        path = os.path.join(base_path, t, mode, video_class)
        video = sorted(os.listdir(path))[video_index]
        data = "video_{}".format(t)
        globals()[data] = []
        for frame in sorted(os.listdir(os.path.join(path, video))):
            globals()[data].append(cv2.cvtColor(
                cv2.imread(
                    os.path.join(path, video, frame)
                ), cv2.COLOR_BGR2RGB
            ))
        globals()[data] = np.r_[globals()[data]]  

    assert video_real.shape[0] == video_fake.shape[0] == video_substraction.shape[0]

    widget = IntSlider(value=32, min=1, max=video_real.shape[0], step=1)

    interact(lambda frame: plt.imshow(video_real[frame-1]), frame=widget)
    interact(lambda frame: plt.imshow(video_fake[frame-1]), frame=widget)
    interact(lambda frame: plt.imshow(video_substraction[frame-1]), frame=widget)
    input("indice actual: {}".format(i))
    clear_output()

interactive(children=(IntSlider(value=32, description='frame', max=64, min=1), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=32, description='frame', max=64, min=1), Output()), _dom_classes=('widge…

interactive(children=(IntSlider(value=32, description='frame', max=64, min=1), Output()), _dom_classes=('widge…

KeyboardInterrupt: Interrupted by user